In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import scipy.stats as st
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from datetime import timedelta
from datetime import time

# Import the OpenWeatherMap API key
from api_keys_weather import api_key

#UFO data files
ufo_data_path = "scrubbed_4_weather.csv"

#read the file
ufo_data = pd.read_csv(ufo_data_path)

# Display the data table for preview
ufo_data

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611
...,...,...,...,...,...,...,...,...,...,...,...
80327,9/9/2013 21:15,nashville,tn,us,light,600.0,10 minutes,Round from the distance/slowly changing colors...,9/30/2013,36.165833,-86.784444
80328,9/9/2013 22:00,boise,id,us,circle,1200.0,20 minutes,Boise&#44 ID&#44 spherical&#44 20 min&#44 10 r...,9/30/2013,43.613611,-116.202500
80329,9/9/2013 22:00,napa,ca,us,other,1200.0,hour,Napa UFO&#44,9/30/2013,38.297222,-122.284444
80330,9/9/2013 22:20,vienna,va,us,circle,5.0,5 seconds,Saw a five gold lit cicular craft moving fastl...,9/30/2013,38.901111,-77.265556


## Use the weather api to retrieve weather data based on the ufo_data

In [2]:
#the 'longitude' column shows "keyerror", check the name 
ufo_data.columns

Index(['datetime', 'city', 'state', 'country', 'shape', 'duration (seconds)',
       'duration (hours/min)', 'comments', 'date posted', 'latitude',
       'longitude '],
      dtype='object')

In [3]:
api_key = "e49190c9dc8b0da7ffc073fae4e34946"


In [4]:
#Longitude data on line 43782 contains erroneous info

# ufo_data_ex = ufo_data.loc[ufo_data["latitude"] != "33q.200088"]
# ufo_data_ex

# #create a dataframe of city, state, country, latitude, longitude

# ufo_location = ufo_data.drop(columns=['shape', 'duration (seconds)', 'duration (hours/min)', 'comments', 'date posted'])
# ufo_location

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611
...,...,...,...,...,...,...,...,...,...,...,...
80327,9/9/2013 21:15,nashville,tn,us,light,600.0,10 minutes,Round from the distance/slowly changing colors...,9/30/2013,36.165833,-86.784444
80328,9/9/2013 22:00,boise,id,us,circle,1200.0,20 minutes,Boise&#44 ID&#44 spherical&#44 20 min&#44 10 r...,9/30/2013,43.613611,-116.202500
80329,9/9/2013 22:00,napa,ca,us,other,1200.0,hour,Napa UFO&#44,9/30/2013,38.297222,-122.284444
80330,9/9/2013 22:20,vienna,va,us,circle,5.0,5 seconds,Saw a five gold lit cicular craft moving fastl...,9/30/2013,38.901111,-77.265556


In [5]:
#change datatime format for url request
#change the pandas datetime format
ufo_time = pd.DataFrame({
    'Date': ufo_data["datetime"]
})
ufo_time['datetime_zero'] = ufo_time['Date'].str.replace('24:00', '0:00')
ufo_time['datetime_er'] = pd.to_datetime(ufo_time['datetime_zero'], format='%m/%d/%Y %H:%M')
selrow = ufo_time['Date'].str.contains('24:00')
ufo_datetime = ufo_time['datetime_obj'] = ufo_time['datetime_er'] + selrow * timedelta(days=1)

# remove time from Date and store it in a new column
ufo_data["Date"] = ufo_datetime.dt.date

# convert time to hourly
ufo_data["Time"]= (ufo_datetime.dt.hour)*100

#info of the ufo_data
# ufo_data.info()

#preview of the ufo_data with new columns of 'Date' and 'Time'
ufo_data

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,Date,Time
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111,1949-10-10,2000
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082,1949-10-10,2100
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667,1955-10-10,1700
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833,1956-10-10,2100
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611,1960-10-10,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80327,9/9/2013 21:15,nashville,tn,us,light,600.0,10 minutes,Round from the distance/slowly changing colors...,9/30/2013,36.165833,-86.784444,2013-09-09,2100
80328,9/9/2013 22:00,boise,id,us,circle,1200.0,20 minutes,Boise&#44 ID&#44 spherical&#44 20 min&#44 10 r...,9/30/2013,43.613611,-116.202500,2013-09-09,2200
80329,9/9/2013 22:00,napa,ca,us,other,1200.0,hour,Napa UFO&#44,9/30/2013,38.297222,-122.284444,2013-09-09,2200
80330,9/9/2013 22:20,vienna,va,us,circle,5.0,5 seconds,Saw a five gold lit cicular craft moving fastl...,9/30/2013,38.901111,-77.265556,2013-09-09,2200


In [6]:
#Longitude data on line 43782 contains erroneous info
ufo_data = ufo_data.loc[ufo_data["latitude"] != "33q.200088",:]

In [7]:
#converting dtypes using astype (Will helped here)
ufo_data['Date'] = ufo_data.Date.astype('str')

In [8]:
#check the dtype (Will helped here)
ufo_data['Date']

0        1949-10-10
1        1949-10-10
2        1955-10-10
3        1956-10-10
4        1960-10-10
            ...    
80327    2013-09-09
80328    2013-09-09
80329    2013-09-09
80330    2013-09-09
80331    2013-09-09
Name: Date, Length: 80331, dtype: object

In [9]:
#converting dtypes using astype to make sure lat & lon match the url request parameters (Will helped here)
ufo_data['longitude '] = ufo_data['longitude '].astype(float)
ufo_data['latitude'] = ufo_data['latitude'].astype(float)


In [10]:
#converting the column 'Date' to datetime format and add a new column 'date' (Will helped here)
ufo_data['date'] = pd.to_datetime(ufo_data.Date)

In [11]:
#creating the new column 'year' for next step (Will helped here)
ufo_data['year'] = ufo_data.date.dt.year

In [12]:
# weatherstack.com only have the historical data from 2009, narrow down the time range for url request (Will helped here)
test = ufo_data[ufo_data['year']>2008]

In [13]:
#create a new list for weather info
weather = []
urls = []

#request info via url
for index, row in test.iterrows():
    lat = round(row["latitude"],4)
    lon = round(row["longitude "],4)
    date = row["Date"]
    url = "https://api.weatherstack.com/historical"
    
#after trying the http version and failed many times, change to python version to make the request (Will helped here)
    params = {
              'access_key': api_key,
              'query': f"{lat},{lon}",
              'historical_date':date,
              'hourly':1,
              'interval':24
                }
#in case there's any interruption when requesting the 80,000+ rows info (Will helped here)
    try:
        api_result = requests.get("https://api.weatherstack.com/historical", params)
        api_response = api_result.json()
        weather.append(api_response['historical'][str(date)])
    except:
        pass
    
    # query_url = f"{url}?access_key={api_key}&query={lat},{lon}&historical_date={date}&hourly=1&interval=24"
    # query_url = f"{url}?access_key={api_key}&query=New York&historical_date={date}&hourly=1&interval=24"

    # urls.append(query_url)
    # ufo_test = requests.get(query_url).json()
    # try: 
    #     df = pd.DataFrame(ufo_test['historical'][date]['hourly'])        
    #     # df['match_index'] = index
    #     weather = pd.concat([weathhttps://api.weatherstack.com/historical??access_key=e49190c9dc8b0da7ffc073fae4e34946&query=36.5950000,-82.1888889&historical_date=1961-10-10&hourly=1&interval=24er,df])
    # except:
    #     pass    

In [15]:
#create a dataframe of the weather info
df = pd.DataFrame(weather)

In [16]:
df

,date,date_epoch,astro,mintemp,maxtemp,avgtemp,totalsnow,sunhour,uv_index,hourly
0,2009-10-10,1255132800,"{'sunrise': '07:13 AM', 'sunset': '06:36 PM', ...",-4,7,1,0.0,11.2,2,"[{'time': 0, 'temperature': 7, 'wind_speed': 1..."
1,2009-10-10,1255132800,"{'sunrise': '07:27 AM', 'sunset': '06:54 PM', ...",5,14,9,0.0,11.3,3,"[{'time': 0, 'temperature': 14, 'wind_speed': ..."
2,2009-10-10,1255132800,"{'sunrise': '06:18 AM', 'sunset': '05:49 PM', ...",2,19,10,0.0,7.2,3,"[{'time': 0, 'temperature': 19, 'wind_speed': ..."
3,2009-10-10,1255132800,"{'sunrise': '07:10 AM', 'sunset': '06:36 PM', ...",16,25,22,0.0,8.6,5,"[{'time': 0, 'temperature': 25, 'wind_speed': ..."
4,2009-10-10,1255132800,"{'sunrise': '06:53 AM', 'sunset': '06:23 PM', ...",14,23,17,0.0,11.6,5,"[{'time': 0, 'temperature': 23, 'wind_speed': ..."
...,...,...,...,...,...,...,...,...,...,...
30564,2013-09-09,1378684800,"{'sunrise': '06:25 AM', 'sunset': '07:03 PM', ...",21,34,26,0.0,11.6,6,"[{'time': 0, 'temperature': 34, 'wind_speed': ..."
30565,2013-09-09,1378684800,"{'sunrise': '07:18 AM', 'sunset': '08:05 PM', ...",13,27,20,0.0,11.6,5,"[{'time': 0, 'temperature': 27, 'wind_speed': ..."
30566,2013-09-09,1378684800,"{'sunrise': '06:46 AM', 'sunset': '07:26 PM', ...",17,35,24,0.0,11.6,6,"[{'time': 0, 'temperature': 35, 'wind_speed': ..."
30567,2013-09-09,1378684800,"{'sunrise': '06:45 AM', 'sunset': '07:27 PM', ...",16,26,21,0.0,10.3,5,"[{'time': 0, 'temperature': 26, 'wind_speed': ..."


In [123]:
# https://api.weatherstack.com/historical?access_key=e49190c9dc8b0da7ffc073fae4e34946&query=35.6527778,-97.4777778&historical_date=2000-09-09&hourly=1&interval=24

{'success': False,
 'error': {'code': 615,
  'type': 'request_failed',
  'info': 'Your API request failed. Please try again or contact support.'}}

In [ ]:
# https://api.weatherstack.com/historical?access_key={api_key}&query=35.6527778,-97.4777778&historical_date=2013-09-09&hourly=1&interval=24

In [74]:
# query_url = f"{url}?access_key={api_key}&query={lat},{lon}&historical_date={date}&hourly=1&interval=24"
# query_url

'https://api.weatherstack.com/historical?access_key=5c808dc84c3b36d961b1012dab43eb76&query=21.4180556,-157.8036111&historical_date=1960-10-10&hourly=1&interval=24'

In [ ]:
# https://api.weatherstack.com/historical?access_key=5c808dc84c3b36d961b1012dab43eb76&query=35.6527778,-97.4777778&historical_date=2013-09-09&hourly=1&interval=24

In [68]:
# query_url

'https://api.weatherstack.com/historical?access_key=5c808dc84c3b36d961b1012dab43eb76&query=21.4180556,-157.8036111&historical_date=1960-10-10&interval=24'

In [58]:
# query_url

'https://api.weatherstack.com/historical?access_key=5c808dc84c3b36d961b1012dab43eb76&query=30.2947222,-82.9841667&historical_date=1966-10-10&interval=24'

In [ ]:
# ufo_test = requests.get(query_url).json()

In [18]:
# date = '2013-09-09'
# frame = pd.DataFrame(ufo_test['historical'][date]['hourly'])
# frame[frame['time'] == time]

,time,temperature,wind_speed,wind_degree,wind_dir,weather_code,weather_icons,weather_descriptions,precip,humidity,...,chanceofremdry,chanceofwindy,chanceofovercast,chanceofsunshine,chanceoffrost,chanceofhightemp,chanceoffog,chanceofsnow,chanceofthunder,uv_index
0,0,27,17,186,S,113,[https://cdn.worldweatheronline.com/images/wsy...,[Clear],0,52,...,0,0,0,0,0,0,0,0,0,1
1,100,26,16,187,S,113,[https://cdn.worldweatheronline.com/images/wsy...,[Clear],0,53,...,0,0,0,0,0,0,0,0,0,1
2,200,25,15,188,S,113,[https://cdn.worldweatheronline.com/images/wsy...,[Clear],0,55,...,0,0,0,0,0,0,0,0,0,1
3,300,24,14,189,S,113,[https://cdn.worldweatheronline.com/images/wsy...,[Clear],0,56,...,0,0,0,0,0,0,0,0,0,1
4,400,23,13,200,SSW,113,[https://cdn.worldweatheronline.com/images/wsy...,[Clear],0,58,...,0,0,0,0,0,0,0,0,0,1
5,500,23,13,211,SSW,113,[https://cdn.worldweatheronline.com/images/wsy...,[Clear],0,60,...,0,0,0,0,0,0,0,0,0,1
6,600,23,12,221,SW,113,[https://cdn.worldweatheronline.com/images/wsy...,[Sunny],0,62,...,0,0,0,0,0,0,0,0,0,6
7,700,25,13,216,SW,113,[https://cdn.worldweatheronline.com/images/wsy...,[Sunny],0,59,...,0,0,0,0,0,0,0,0,0,6
8,800,27,13,211,SSW,113,[https://cdn.worldweatheronline.com/images/wsy...,[Sunny],0,56,...,0,0,0,0,0,0,0,0,0,7
9,900,29,14,207,SSW,113,[https://cdn.worldweatheronline.com/images/wsy...,[Sunny],0,54,...,0,0,0,0,0,0,0,0,0,7


In [8]:
# #use testing url to create a dataframe
# url = ('https://api.weatherstack.com/historical?access_key=5c808dc84c3b36d961b1012dab43eb76&query=35.6527778,-97.4777778&historical_date=2013-09-09&hourly=1&interval=1')
# df_test = pd.read_json(url)
# df_test

,request,location,current,historical
type,LatLon,NaN,NaN,NaN
query,Lat 35.65 and Lon -97.48,NaN,NaN,NaN
language,en,NaN,NaN,NaN
unit,m,NaN,NaN,NaN
name,NaN,Edmond,NaN,NaN
country,NaN,United States of America,NaN,NaN
region,NaN,Oklahoma,NaN,NaN
lat,NaN,35.653,NaN,NaN
lon,NaN,-97.478,NaN,NaN
timezone_id,NaN,America/Chicago,NaN,NaN


In [9]:
# detail_weateher_test = df_test["historical"].dropna()
# detail_weateher_test

2013-09-09    {'date': '2013-09-09', 'date_epoch': 137868480...
Name: historical, dtype: object

In [13]:
# ufo_weather_data = []

# for index, row in ufo_data.iterrows():
#     lat = row["latitude"]
#     lon = row["longitude "]
#     date = row["Date"]
#     time = row["Time"]
    
#     weather_data = requests.get(query_url)
#     weather_data = weather_data.json()
    
#     ufo_city_name = weather_data["location"]["name"]
#     ufo_country = weather_data["location"]["country"]
#     ufo_region = weather_data["location"]["region"]
#     ufo_lat = weather_data["location"]["lat"]
#     ufo_lng = weather_data["location"]["lon"]
#     ufo_timezone = weather_data["location"]["timezone_id"]
#     ufo_historical = weather_data["historical"]

In [ ]:
# #create a list to collect weather data

# ufo_weather_data = []

# #Set the API base URL

# for index, row in ufo_data.iterrows():
#     lat = row["latitude"]
#     lon = row["longitude "]
#     date = row["Date"]
#     time = row["Time"]

#     url = "https://api.weatherstack.com/historical"
#     query_url = f"{url}?access_key={api_key}&query={lat},{lon}&historical_date={date}&hourly=1&interval=1"
    
     
#     # Run an API request for each of the cities
#     try:
# #         # Parse the JSON and retrieve data
#         weather_data = requests.get(query_url)
#         weather_data = weather_data.json()
        
# #         # Parse out: 
# #         #city name, country, slatitude, longitude, date, time, timezone
# #         #temperature, wind_speed, weather_code, weather_descriptions
# #         #humidity, visibility, pressure, cloudcover, heatindex, dewpoint
#         ufo_city_name = weather_data["location"]["name"]
#         ufo_country = weather_data["location"]["country"]
#         ufo_region = weather_data["location"]["region"]
#         ufo_lat = weather_data["location"]["lat"]
#         ufo_lng = weather_data["location"]["lon"]
#         ufo_timezone = weather_data["location"]["timezone_id"]
#         ufo_historical = weather_data["historical"]

# #         # Append the City information into city_data list
# #"City": ufo_city_name, 
#                           # "Lat": ufo_lat, 
#                           # "Lng": ufo_lng, 
#                           # "Timezone": ufo_timezone,
#                           # "Country": ufo_country,
#                           # "Region": ufo_region,

#         ufo_weather_data.append({
#                           "Historical Data": ufo_historical,
#                           })

# # #     # If an error is experienced, skip the city
#     except:
#         print("info not found. Skipping...")
#         pass
              
# # # # Indicate that Data Loading is complete 
# print("-----------------------------")
# print("Data Retrieval Complete      ")
# print("-----------------------------")

info not found. Skipping...
info not found. Skipping...
info not found. Skipping...
info not found. Skipping...
info not found. Skipping...
info not found. Skipping...
info not found. Skipping...
info not found. Skipping...
info not found. Skipping...
info not found. Skipping...
info not found. Skipping...
info not found. Skipping...
info not found. Skipping...
info not found. Skipping...
info not found. Skipping...
info not found. Skipping...
info not found. Skipping...
info not found. Skipping...
info not found. Skipping...
info not found. Skipping...
info not found. Skipping...
info not found. Skipping...
info not found. Skipping...
info not found. Skipping...
info not found. Skipping...
info not found. Skipping...
info not found. Skipping...
info not found. Skipping...
info not found. Skipping...


In [19]:
# Export the weather_data into a csv

df.to_csv("ufo_weathers_info.csv", index_label="index")


In [27]:
# merge ufo_data and weather_data 
#pd.concat([df1,df2])

ufo_weather = pd.concat([ufo_data, df], axis=1)

In [29]:
# Export the ufo_weather into a csv
ufo_weather.to_csv("detailed_ufo_weather.csv", index_label = "index")

## Create the Scatter Plots
### Latitude Vs. Average Temperature

In [ ]:
# Build scatter plot for latitude vs. temperature
lat_plot = ufo_weather["Lat"]
temp_plot = ufo_weather["Max Temp"]